In [ ]:
sc

Out[1]:

SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
from pyspark import SparkFiles
from  pyspark.ml.feature import StopWordsRemover, RegexTokenizer, HashingTF, StringIndexer, IDF
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark import SparkFiles
import pandas as pd
from io import StringIO
import boto3
from botocore.client import Config

In [ ]:
def write_output_to_aws(output_file_name, output_predictions):
  ACCESS_KEY = ''
  SECRET_KEY = ''
  AWS_BUCKET_NAME = ""
  s3 = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key =SECRET_KEY, config = Config(signature_version = 's3v4') )
  op = pd.DataFrame(output_predictions)
  op.columns = ['Tweet','probabilities', 'prediction', 'true_label']
  csv_buffer = StringIO()
  op.to_csv(csv_buffer)
  # s3_resource = boto3.resource('s3')
#   timestamp = int(time.time())
  file_name = f'output_new/{output_file_name}.csv'
  s3.Object(AWS_BUCKET_NAME, file_name).put(Body=csv_buffer.getvalue(),ACL='public-read')
#   you can change the region or else add the region dynamically
  region= "us-west-1"

  url = f"https://{AWS_BUCKET_NAME}.s3.{region}.amazonaws.com/{file_name}"
  print("------------------------------------")
  print("download url from aws: ", url)
  print("------------------------------------")

In [ ]:
def train_and_get_tweets_predictions(url, output_file_name):
  spark.sparkContext.addFile(url)
  # df = spark.read.csv('/FileStore/tables/Tweets.csv', header= True, inferSchema= True)
#   Loading the file
  df = spark.read.csv("file://"+SparkFiles.get('Tweets.csv'), header= True, inferSchema= True)
#   Selecting only tweets and sentiment
  data = df.select(['airline_sentiment','text'])
#   Removing empty rows if any
  data = data.where(data.text.isNotNull())


# initializing tokenizers, stopwords_remover, hashingTF, string indexer and Logistic Regression
  tokenizer = RegexTokenizer(pattern=r'(?:\p{Punct}|\s)+', inputCol='text', outputCol='text_temp')
  remover = StopWordsRemover(inputCol='text_temp', outputCol='text_intermediate')
  hashing_tf_words = HashingTF(inputCol="text_intermediate", outputCol="features")
  string_indexer = StringIndexer(inputCol='airline_sentiment', outputCol='label')
  lr = LogisticRegression(regParam=0.3, elasticNetParam=0, maxIter = 10000)
# fitting them in a pipeline
  pipeline = Pipeline(stages=[tokenizer, remover, hashing_tf_words, string_indexer, lr])
# building a parameter grid for hypertuning
  paramGrid = ParamGridBuilder() \
      .addGrid(hashing_tf_words.numFeatures, [10, 100, 1000]) \
      .addGrid(lr.regParam, [0.2, 0.4, 0.6, 0.8]) \
      .addGrid(lr.maxIter, [100,1000,10000]) \
      .build()

#   crossvalidator
  crossval = CrossValidator(estimator=pipeline,
                            estimatorParamMaps=paramGrid,
                            evaluator=MulticlassClassificationEvaluator(),
                            numFolds=2,
                            )

  train, test = data.randomSplit([0.8, 0.2], seed = 1)
  cvModel = crossval.fit(train)
  prediction = cvModel.transform(test)

#   evaluating the accuracy
  evaluator = MulticlassClassificationEvaluator(metricName = "accuracy")
  accuracy = evaluator.evaluate(prediction)
  print("------------------------")
  print("accuracy is :",accuracy)
  print("------------------------")
  output_to_show = prediction.select(["text", "probability","prediction","label"])
  write_output_to_aws(output_file_name, output_to_show.collect())

In [ ]:
if __name__=="__main__":
# bucket url
  url = ""
  output_file_name = "output_tweets"
  train_and_get_tweets_predictions(url, output_file_name)

/databricks/spark/python/pyspark/ml/util.py:838: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
------------------------
accuracy is : 0.737874097007224
------------------------
------------------------------------
download url from aws: https://bigdataproject.s3.us-west-1.amazonaws.com/output_new/output_tweets.csv
------------------------------------